## ExampleSelector
- Few-shot 프롬프트를 위해 예시를 제공할 때, 일부 예시만 사용해야할 수 있음.
    - 예) token 제약으로, 모든 sample을 제공할 수 없을 경우

- 따라서, 특정 조건을 만족하는 sample example만 Few-shot 프롬프트에 사용할 수 있음.
    - Langchain에서 제공하는 selector를 사용
    - 또는, BaseExampleSelector를 상속받아, selector를 커스터마이징하여 사용

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

In [2]:
chat = ChatOpenAI(
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

## Template version

In [3]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = PromptTemplate.from_template(
    "Human: {question}\nAI: {answer}"
)

example_selector = LengthBasedExampleSelector(
    example_prompt=example_template,   # 프롬프트 템플릿
    examples=examples,   # Few-shot 프롬프트 예시
    max_length=1000   # 토큰 길이 제한
)

prompt = FewShotPromptTemplate(
    example_prompt=example_template,
    example_selector=example_selector,  # example 대신, example selector 객체를 주입!
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## Chat version

In [4]:
# chat version

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = ChatPromptTemplate.from_messages(
    messages=[
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_few_shot_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_template
)

prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "You are a geography expert. You can give a long answer"),
        example_few_shot_template,
        ("human", "What do you know about {country}?")
    ]
)

chain = prompt | chat

chain.invoke({"country": "Thailand"})

Thailand is a country located in Southeast Asia known for its rich culture, stunning beaches, and delicious cuisine. Here are some key facts about Thailand:

Capital: Bangkok
Language: Thai
Currency: Thai Baht
Famous for: Temples, tropical islands, vibrant street markets, and spicy and flavorful cuisine
Tourist attractions: The Grand Palace, Wat Pho, Ayutthaya Historical Park, Phi Phi Islands, Chiang Mai Night Bazaar
Climate: Tropical climate with a rainy season from May to October and a dry season from November to April
Culture: Known for its traditional dance, music, and festivals such as Songkran (Thai New Year) and Loy Krathong (Festival of Lights)
Economy: Thailand's economy is heavily reliant on tourism, agriculture, and manufacturing, with exports including rice, textiles, and electronics
Overall, Thailand is a diverse and beautiful country that offers a mix of cultural experiences, natural beauty, and delicious food for visitors to enjoy.

AIMessageChunk(content="Thailand is a country located in Southeast Asia known for its rich culture, stunning beaches, and delicious cuisine. Here are some key facts about Thailand:\n\nCapital: Bangkok\nLanguage: Thai\nCurrency: Thai Baht\nFamous for: Temples, tropical islands, vibrant street markets, and spicy and flavorful cuisine\nTourist attractions: The Grand Palace, Wat Pho, Ayutthaya Historical Park, Phi Phi Islands, Chiang Mai Night Bazaar\nClimate: Tropical climate with a rainy season from May to October and a dry season from November to April\nCulture: Known for its traditional dance, music, and festivals such as Songkran (Thai New Year) and Loy Krathong (Festival of Lights)\nEconomy: Thailand's economy is heavily reliant on tourism, agriculture, and manufacturing, with exports including rice, textiles, and electronics\nOverall, Thailand is a diverse and beautiful country that offers a mix of cultural experiences, natural beauty, and delicious food for visitors to enjoy.")

## Customize ExampleSelector

In [12]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    # must
    def add_example(self, example):
        self.examples.append(example)    
        
    # must
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [13]:
example_selector = RandomExampleSelector(
    examples=examples,
)

In [14]:
prompt = FewShotPromptTemplate(
    example_prompt=example_template,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'